In [12]:
import google.generativeai as genai
from google.api_core import retry
from IPython.display import display, Markdown
import re

api_key = "Your api key"

genai.configure(api_key=api_key)

# Order-related functions (for managing poem generation)
def generate_poem(poem_type, theme, tone, emotion) -> str:
    """Generates a poem based on the input specifications."""
    user_prompt = f"""
    Write a {poem_type} that explores the theme of {theme}. 
    The poem should be divided into stanzas 
    follow the rules of the {poem_type} conditions like structure of the poem, number of lines for that poem type.
    Maintain a {tone} tone throughout. 
    Choose words intentionally for {emotion} to evoke a strong emotional response from the reader.
    Rules:
    --donot generate an
    """
    model_name = 'gemini-2.0-flash'
    
    # Initialize the model with the prompt and options
    model = genai.GenerativeModel(model_name, system_instruction=user_prompt)
    
    # Start the conversation
    convo = model.start_chat(enable_automatic_function_calling=True)
    
    # Request poem generation from Gemini
    response = model.generate_content(user_prompt)
    
    response = response.text.replace('*', '').replace('#', '')
    return response, convo

def capture_user_inputs():
    """Captures user inputs for the poem generation."""
    print("Welcome to the Poem Generator!")
    
    poem_type = input("Enter the type of poem (e.g., sonnet, haiku, limerick, Free Verse, Ballad, Acrostic, Ode, Villanelle, Epic, Tanka, Cinquain, concrete poetry): ")
    theme = input("Enter the theme of the poem (e.g., Love, Nature, Loss, Friendship, Life, Death, War, Peace, Hope, Despair, Happiness, Sorrow, Beauty, Freedom, Betrayal, Family, Courage, Time, Change, Identity, Memory, Youth, Aging, Fear, Justice, Equality, Isolation, Sacrifice, Grief, Faith, Redemption, Power, Conflict, Society, Dreams, Ambition, Hope, Truth): ")
    tone = input("Enter the tone of the poem (e.g., Romantic, Melancholic, Joyful, Mournful, Nostalgic, Optimistic, Pessimistic, Reflective, Angry, Calm, Confessional, Inspirational, Satirical, Playful, Dark, Spiritual, Tragic, Eerie, Philosophical, Surreal): ")
    emotion = input("Enter the emotion you want to evoke (e.g., Joy, Sadness, Anger, Fear, Love, Hope, Grief, Confusion, Nostalgia, Surprise, Excitement, Loneliness, Despair, Peace, Anxiety, Passion, Contentment, Envy, Empathy, Regret, Guilt, Pride, Relief, Awe, Compassion, Resentment, Sympathy, Melancholy, Euphoria, Resilience, Vulnerability, Frustration, Gratitude): ")
    
    return poem_type, theme, tone, emotion

# Function to send message to the model and get the response
@retry.Retry(initial=30)
def send_message(convo, message):
    return convo.send_message(message)

def qa_on_poem(poem, user_question, convo):
    """Generates an answer to a question based on the generated poem."""
    qa_prompt = f"Here is a poem: {poem}\nAnswer the following question about the poem: {user_question}"
    
    model_name = 'gemini-2.0-flash'
    model = genai.GenerativeModel(model_name, system_instruction=f"{qa_prompt} You are a QA bot that answers questions based on the poem provided.")
    
    response = model.generate_content(qa_prompt)
    response = response.text.replace('*', '').replace('#', '')
    
    return response, convo

# Initialize the ordering system for poem generation and QA bot
def main():
    # Initialize conversation with the model
    model_name = 'gemini-2.0-flash'
    poem_tools = [generate_poem]
    
    model = genai.GenerativeModel(
        model_name, tools=poem_tools, system_instruction="You are a poem generator bot. Follow the user’s instructions to generate poems based on their preferences.")
    
    convo = model.start_chat(enable_automatic_function_calling=True)

    while True:
        # Capture user inputs for poem generation
        poem_type, theme, tone, emotion = capture_user_inputs()

        # Generate the poem
        poem, convo = generate_poem(poem_type, theme, tone, emotion)
        print("\nGenerated Poem:\n", poem)

        # Allow user to ask questions about the poem
        while True:
            user_question = input("\nAsk a question about the poem (or type 'exit' to stop asking questions): ")
            if user_question.lower() == 'exit':
                break
            
            answer, convo = qa_on_poem(poem, user_question, convo)
            display(Markdown(answer))
        
        # Ask the user if they want to generate another poem
        if input("\nDo you want to generate another poem? (yes/no): ").lower() != 'yes':
            break

if __name__ == "__main__":
    main()


Welcome to the Poem Generator!


Enter the type of poem (e.g., sonnet, haiku, limerick, Free Verse, Ballad, Acrostic, Ode, Villanelle, Epic, Tanka, Cinquain, concrete poetry):  Acrostic
Enter the theme of the poem (e.g., Love, Nature, Loss, Friendship, Life, Death, War, Peace, Hope, Despair, Happiness, Sorrow, Beauty, Freedom, Betrayal, Family, Courage, Time, Change, Identity, Memory, Youth, Aging, Fear, Justice, Equality, Isolation, Sacrifice, Grief, Faith, Redemption, Power, Conflict, Society, Dreams, Ambition, Hope, Truth):   Friendship
Enter the tone of the poem (e.g., Romantic, Melancholic, Joyful, Mournful, Nostalgic, Optimistic, Pessimistic, Reflective, Angry, Calm, Confessional, Inspirational, Satirical, Playful, Dark, Spiritual, Tragic, Eerie, Philosophical, Surreal):  Inspirational
Enter the emotion you want to evoke (e.g., Joy, Sadness, Anger, Fear, Love, Hope, Grief, Confusion, Nostalgia, Surprise, Excitement, Loneliness, Despair, Peace, Anxiety, Passion, Contentment, Envy, Empathy, Regret, Guilt, Pride, R


Generated Poem:
 Okay, here's an acrostic poem about Friendship, divided into stanzas, inspirational in tone, and aiming to evoke a strong emotional response, while adhering to your specifications:

Faithful hearts, a bond so true,
Radiant smiles shared between two.

Inspiring dreams, together we chase,
Encouraging words, filling every space.

Nurturing souls, a gentle hand,
Devotion shown across the land.

Support unwavering, through thick and thin,
Harmony found, where love begins.
In moments of joy, or times of despair,
Precious treasure, beyond compare.




Ask a question about the poem (or type 'exit' to stop asking questions):  where was love begins


According to the poem, love begins where harmony is found.



Ask a question about the poem (or type 'exit' to stop asking questions):  exit'


This query is incomplete. Please provide the question about the poem that you would like me to answer.



Ask a question about the poem (or type 'exit' to stop asking questions):  exit

Do you want to generate another poem? (yes/no):  no
